# Extract Citizens auto table

By: Mohamed Al Elew

In [1]:
import os
import re
import zipfile

import geopandas as gpd
import pandas as pd
import stringcase
import tabula

# Constants

In [2]:
RATE_TABLE_PDF_FP = "./inputs/territory_base_rates.pdf"
TERRITORY_TABLE_PDF_FP = "./inputs/territory_definitions.pdf"
META_DATA_FP = "./inputs/auto_rate_coverage_meta_data.csv"

UNPROCESSED_EXPORT_FP = "./outputs/auto_rate_table.csv"
EXPORT_FP = "./outputs/allstate_auto.csv"
MAP_EXPORT_FP = "./outputs/allstate_auto_gis.geojson"

In [3]:
COMPANY_NAME = "AllState"
VEHICLE_TYPE = "Auto"
FACTOR_CIRCUMVENTED = "zip"
FACTOR_NAME = "Territorial Base Rate"
GEOGRAPHY_FACTOR = "Territory"
GEOGRAPHY_TYPE = "zip"

In [4]:
RATE_TABLE_COLS = [
    "territory",
    "bodily_injury",
    "property_damage",
    "pip",
    "collision",
    "comprehensive",
    "um_single_car",
    "um_multi_car",
    "uim_single_car",
    "uim_multi_car",
]

RATE_UM_UIM_COLS = [
    "rate_um_single_car",
    "rate_um_multi_car",
    "rate_uim_single_car",
    "rate_uim_multi_car",
]

BASE_COLUMNS = [
    "company",
    "vehicle_type",
    "factor_circumvented",
    "factor_name",
    "geography_factor",
    "geography_type",
    "geography_factor_id",
    "latitude",
    "longitude",
    "county_fips",
    "tract_fips",
    "block_group_fips",
    "zip",
    "place_name_fips",
]

GIS_COLS = [
    "geo_id",
    "geo_name",
    "tot_pop",
    "white_tot",
    "black_tot",
    "latin_tot",
    "white_pct",
    "black_pct",
    "latin_pct",
    "median_income",
    "geometry",
]

In [5]:
def clean_column_name(col):
    return col.replace("\r", " ").replace("/", " ").replace(" ", "_").lower()

In [6]:
RATE_PREFIX = "rate_"


def get_rate_columns(df_rate_table):
    return [col for col in df_rate_table.columns if col.startswith(RATE_PREFIX)]


def get_avg_rate_factor(df_rate_table):
    rate_cols = get_rate_columns(df_rate_table)
    return df_rate_table[rate_cols].mean(axis=1)

In [7]:
def read_table(pdf_fp, page_list, **kwargs):
    col2str = {"dtype": str}
    kwargs = {"pandas_options": col2str}
    tables = tabula.read_pdf(pdf_fp, pages=page_list, **kwargs)
    return pd.concat([table for table in tables])


def read_table_in_batches(pdf_fp, page_count, batch_step):
    tables = []
    page_steps = list(range(1, page_count, batch_step)) + [page_count]
    for idx, page in enumerate(page_steps):
        if idx + 1 < len(page_steps):
            pages = list(range(page, page_steps[idx + 1]))
            tables.append(read_table(pdf_fp, pages))
    return tables

# Read territory table

In [8]:
DF_TERRITORY_TABLE = read_table(TERRITORY_TABLE_PDF_FP, "all")
DF_TERRITORY_TABLE

,COUNTY,COUNTY CODE,TERRITORY,ZIP CODE,TOWN,AREA
0,SAINT CLAIR,74,8001,48001,ALGONAC,700
1,SAINT CLAIR,74,8001,48001,CLAY,700
2,SAINT CLAIR,74,8001,48001,PEARL BEACH,700
3,SAINT CLAIR,74,8002,48002,ALLENTON,700
4,SAINT CLAIR,74,8002,48002,BERLIN,700
...,...,...,...,...,...,...
101,HOUGHTON,31,9967,49967,KENTON,180
102,GOGEBIC,27,9968,49968,WAKEFIELD,150
0,GOGEBIC,27,9969,49969,WATERSMEET,150
1,BARAGA,7,9970,49970,WATTON,190


# Read base rate table

In [9]:
DF_RATE_TABLE = read_table(RATE_TABLE_PDF_FP, "all")
DF_RATE_TABLE

,TERRITORY,Bodily Injury\r(AA)\r20/40,"Property Damage\r(BB/BV)\r$100,000",PIP\r(VA-01),Collision\r(DD)\r$100 DED.,Comprehensive\r(HH)\r$50 DED.,UM\r(SS)\r20/40,UIM\r(SU)\r20/40,Unnamed: 0,Unnamed: 1
0,NaN,NaN,NaN,NaN,NaN,Single\rCar,Multi-\rCar,Single\rCar,Multi-\rCar,NaN
1,8001,442.38,165.41,"6,709.55",967.49,104.23,28.70,28.70,30.21,30.21
2,8002,527.19,139.70,"6,845.72",866.26,107.10,28.06,28.06,30.13,30.13
3,8003,527.19,139.70,"6,845.72",866.26,107.10,28.06,28.06,30.13,30.13
4,8004,488.94,165.41,"6,709.55",1055.18,104.23,28.70,28.70,30.21,30.21
...,...,...,...,...,...,...,...,...,...,...
26,9967,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
27,9968,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
28,9969,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
29,9970,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02


# Build processed table

In [10]:
df_processed_table = DF_RATE_TABLE.copy()
df_processed_table

,TERRITORY,Bodily Injury\r(AA)\r20/40,"Property Damage\r(BB/BV)\r$100,000",PIP\r(VA-01),Collision\r(DD)\r$100 DED.,Comprehensive\r(HH)\r$50 DED.,UM\r(SS)\r20/40,UIM\r(SU)\r20/40,Unnamed: 0,Unnamed: 1
0,NaN,NaN,NaN,NaN,NaN,Single\rCar,Multi-\rCar,Single\rCar,Multi-\rCar,NaN
1,8001,442.38,165.41,"6,709.55",967.49,104.23,28.70,28.70,30.21,30.21
2,8002,527.19,139.70,"6,845.72",866.26,107.10,28.06,28.06,30.13,30.13
3,8003,527.19,139.70,"6,845.72",866.26,107.10,28.06,28.06,30.13,30.13
4,8004,488.94,165.41,"6,709.55",1055.18,104.23,28.70,28.70,30.21,30.21
...,...,...,...,...,...,...,...,...,...,...
26,9967,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
27,9968,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
28,9969,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
29,9970,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02


## Drop garbage text

In [11]:
is_header_garbage_text = (
    df_processed_table[df_processed_table.columns[:4]].isna().any(axis=1)
)
df_processed_table = df_processed_table[~is_header_garbage_text].copy()

## Set column names

In [12]:
df_processed_table.columns = RATE_TABLE_COLS
df_processed_table

,territory,bodily_injury,property_damage,pip,collision,comprehensive,um_single_car,um_multi_car,uim_single_car,uim_multi_car
1,8001,442.38,165.41,"6,709.55",967.49,104.23,28.70,28.70,30.21,30.21
2,8002,527.19,139.70,"6,845.72",866.26,107.10,28.06,28.06,30.13,30.13
3,8003,527.19,139.70,"6,845.72",866.26,107.10,28.06,28.06,30.13,30.13
4,8004,488.94,165.41,"6,709.55",1055.18,104.23,28.70,28.70,30.21,30.21
5,8005,501.11,150.35,"7,173.95",957.45,107.10,30.99,30.99,33.30,33.30
...,...,...,...,...,...,...,...,...,...,...
26,9967,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
27,9968,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
28,9969,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02
29,9970,351.34,163.31,"6,620.55",646.77,140.37,15.76,15.76,17.02,17.02


In [13]:
def create_rate_label(col_name):
    return RATE_PREFIX + col_name


rate_col_names = df_processed_table.columns[1:]
df_processed_table[rate_col_names] = df_processed_table[rate_col_names].replace(
    ",", "", regex=True
)
df_processed_table[rate_col_names] = df_processed_table[rate_col_names].astype(float)
extracted_rate_col_names = [create_rate_label(col_name) for col_name in rate_col_names]
rate_col_renames = dict(zip(rate_col_names, extracted_rate_col_names))
df_processed_table = df_processed_table.rename(columns=rate_col_renames)
df_processed_table

,territory,rate_bodily_injury,rate_property_damage,rate_pip,rate_collision,rate_comprehensive,rate_um_single_car,rate_um_multi_car,rate_uim_single_car,rate_uim_multi_car
1,8001,442.38,165.41,6709.55,967.49,104.23,28.70,28.70,30.21,30.21
2,8002,527.19,139.70,6845.72,866.26,107.10,28.06,28.06,30.13,30.13
3,8003,527.19,139.70,6845.72,866.26,107.10,28.06,28.06,30.13,30.13
4,8004,488.94,165.41,6709.55,1055.18,104.23,28.70,28.70,30.21,30.21
5,8005,501.11,150.35,7173.95,957.45,107.10,30.99,30.99,33.30,33.30
...,...,...,...,...,...,...,...,...,...,...
26,9967,351.34,163.31,6620.55,646.77,140.37,15.76,15.76,17.02,17.02
27,9968,351.34,163.31,6620.55,646.77,140.37,15.76,15.76,17.02,17.02
28,9969,351.34,163.31,6620.55,646.77,140.37,15.76,15.76,17.02,17.02
29,9970,351.34,163.31,6620.55,646.77,140.37,15.76,15.76,17.02,17.02


## Aggregate values

In [14]:
df_processed_table

,territory,rate_bodily_injury,rate_property_damage,rate_pip,rate_collision,rate_comprehensive,rate_um_single_car,rate_um_multi_car,rate_uim_single_car,rate_uim_multi_car
1,8001,442.38,165.41,6709.55,967.49,104.23,28.70,28.70,30.21,30.21
2,8002,527.19,139.70,6845.72,866.26,107.10,28.06,28.06,30.13,30.13
3,8003,527.19,139.70,6845.72,866.26,107.10,28.06,28.06,30.13,30.13
4,8004,488.94,165.41,6709.55,1055.18,104.23,28.70,28.70,30.21,30.21
5,8005,501.11,150.35,7173.95,957.45,107.10,30.99,30.99,33.30,33.30
...,...,...,...,...,...,...,...,...,...,...
26,9967,351.34,163.31,6620.55,646.77,140.37,15.76,15.76,17.02,17.02
27,9968,351.34,163.31,6620.55,646.77,140.37,15.76,15.76,17.02,17.02
28,9969,351.34,163.31,6620.55,646.77,140.37,15.76,15.76,17.02,17.02
29,9970,351.34,163.31,6620.55,646.77,140.37,15.76,15.76,17.02,17.02


In [15]:
assert df_processed_table["rate_um_multi_car"].equals(
    df_processed_table["rate_um_single_car"]
)
assert df_processed_table["rate_uim_multi_car"].equals(
    df_processed_table["rate_uim_single_car"]
)
df_processed_table["rate_um_uim"] = (
    df_processed_table["rate_um_single_car"] + df_processed_table["rate_uim_single_car"]
)
df_processed_table = df_processed_table.drop(RATE_UM_UIM_COLS, axis=1)

In [16]:
df_processed_table[get_rate_columns(df_processed_table)] = df_processed_table[
    get_rate_columns(df_processed_table)
].astype(float)
df_processed_table["generic_location_based_premium"] = df_processed_table[
    get_rate_columns(df_processed_table)
].sum(axis=1)
df_processed_table

,territory,rate_bodily_injury,rate_property_damage,rate_pip,rate_collision,rate_comprehensive,rate_um_uim,generic_location_based_premium
1,8001,442.38,165.41,6709.55,967.49,104.23,58.91,8447.97
2,8002,527.19,139.70,6845.72,866.26,107.10,58.19,8544.16
3,8003,527.19,139.70,6845.72,866.26,107.10,58.19,8544.16
4,8004,488.94,165.41,6709.55,1055.18,104.23,58.91,8582.22
5,8005,501.11,150.35,7173.95,957.45,107.10,64.29,8954.25
...,...,...,...,...,...,...,...,...
26,9967,351.34,163.31,6620.55,646.77,140.37,32.78,7955.12
27,9968,351.34,163.31,6620.55,646.77,140.37,32.78,7955.12
28,9969,351.34,163.31,6620.55,646.77,140.37,32.78,7955.12
29,9970,351.34,163.31,6620.55,646.77,140.37,32.78,7955.12


## Calculated weighted mean

In [17]:
# pulled from this filing: https://www.documentcloud.org/documents/24531812-3r29010-filing-memo-allstate-pre-reform-earned-totals-alse-130457191#document/p14/a2551187
%run ../../00_misc/helper_func_notebook.ipynb

weights = read_weights(META_DATA_FP)
weights

Exception: File `'../../00_misc/helper_func_notebook.ipynb'` not found.

In [ ]:
df_processed_table["weighted_mean"] = calculate_weighted_mean(
    df_processed_table[get_rate_columns(df_processed_table)], weights
)

In [ ]:
median_loc_base_rate = df_processed_table["generic_location_based_premium"].median()
df_processed_table["location_effect"] = (
    df_processed_table["generic_location_based_premium"] / median_loc_base_rate
)

## Geography columns

Rename geography index column

In [ ]:
df_processed_table = df_processed_table.rename(
    columns={"territory": "geography_factor_id"}
)
df_processed_table

## Fill constants

In [ ]:
df_processed_table["company"] = COMPANY_NAME
df_processed_table["vehicle_type"] = VEHICLE_TYPE
df_processed_table["factor_circumvented"] = FACTOR_CIRCUMVENTED
df_processed_table["factor_name"] = FACTOR_NAME
df_processed_table["geography_factor"] = GEOGRAPHY_FACTOR
df_processed_table["geography_type"] = GEOGRAPHY_TYPE

In [ ]:
def fill_null_columns(df, base_columns):
    base_col_to_fill = list(set(base_columns) - set(df.columns))
    df[base_col_to_fill] = None
    return df


df_processed_table = fill_null_columns(df_processed_table, BASE_COLUMNS)

# Zip code territory mappings

## Read data

In [ ]:
df_trty_table = DF_TERRITORY_TABLE.copy()
df_trty_table.columns = [clean_column_name(col) for col in df_trty_table.columns]

In [ ]:
df_trty_zip_table = df_trty_table[["territory", "zip_code"]].copy()
df_trty_zip_table = df_trty_zip_table.drop_duplicates()

## Merge zip code and rate table

In [ ]:
df_processed_table.sort_values(["geography_factor_id"])

In [ ]:
df_processed_table = df_processed_table.merge(
    df_trty_zip_table,
    how="left",
    validate="1:1",
    left_on="geography_factor_id",
    right_on="territory",
)

In [ ]:
df_processed_table

# Demographic and GIS data

## Read data

In [ ]:
GDF_ZIP = gpd.read_file(
    "../../01_demographics/outputs/zcta_2019_demographics_map.geojson"
)
gdf_zip = GDF_ZIP.copy()
gdf_zip = gdf_zip[GIS_COLS].copy()
gdf_zip.head()

In [ ]:
assert gdf_zip["white_tot"].sum() > gdf_zip["black_tot"].sum()

## Merge

In [ ]:
zip_len = len(df_processed_table["zip_code"][1])
gdf_zip["geo_id"] = gdf_zip["geo_id"].str.slice(-1 * zip_len)

In [ ]:
gdf_processed_table = gdf_zip.merge(
    df_processed_table,
    right_on="zip_code",
    left_on="geo_id",
    how="right",
    validate="1:1",
)
gdf_processed_table

In [ ]:
gdf_processed_table.info()

## Update data types

In [ ]:
gdf_processed_table = gdf_processed_table.replace("-", 0)
gdf_processed_table["median_income"] = gdf_processed_table["median_income"].replace(
    "2,500-", "2500"
)
gdf_processed_table["median_income"] = gdf_processed_table["median_income"].astype(
    float
)

## Detroit filter

In [ ]:
MUNICIPAL_BOUND_GIS_FP = "../../02_allstate/inputs/Municipal_Boundaries.zip"
MUNICIPALITIES_TO_EXTRACT = ["Detroit"]
GDF_MUNICIPAL = gpd.read_file(MUNICIPAL_BOUND_GIS_FP)
gdf_detroit = GDF_MUNICIPAL[GDF_MUNICIPAL["NAME"].isin(MUNICIPALITIES_TO_EXTRACT)]

In [ ]:
gdf_detroit_points = list(
    gpd.overlay(gdf_processed_table, gdf_detroit.to_crs("EPSG:4269"))[
        "geography_factor_id"
    ]
)
gdf_processed_table["is_in_detroit"] = gdf_processed_table["geography_factor_id"].isin(
    gdf_detroit_points
)

In [ ]:
gdf_processed_table[gdf_processed_table["is_in_detroit"]].plot()

# Export

In [ ]:
DF_RATE_TABLE.to_csv("./outputs/auto_rate_table.csv", index=False)

In [ ]:
df_processed_table.to_csv("./outputs/allstate_auto.csv", index=False)

In [ ]:
gdf_processed_table.to_file("./outputs/allstate_auto_gis.geojson", driver="GeoJSON")
gdf_processed_table.info()

# Appendix

I need to come back and verify this but these are likely specialized zip codes for building and institutions that receive a lot of mail.

In [ ]:
gdf_processed_table[gdf_processed_table["geo_id"].isnull()][
    "zip_code"
]  # .to_csv("test.csv")